In [1]:
#import Libraries
import numpy as np
import pandas as pd
from sklearn import datasets,preprocessing,model_selection,linear_model,metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
data=pd.read_csv("/content/train.csv")
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [ ]:
dataset = data[['LotArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','TotRmsAbvGrd','SalePrice']]
dataset

,LotArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,TotRmsAbvGrd,SalePrice
0,8450,1,0,2,1,8,208500
1,9600,0,1,2,0,6,181500
2,11250,1,0,2,1,6,223500
3,9550,1,0,1,0,7,140000
4,14260,1,0,2,1,9,250000
...,...,...,...,...,...,...,...
1455,7917,0,0,2,1,7,175000
1456,13175,1,0,2,0,7,210000
1457,9042,0,0,2,0,9,266500
1458,9717,1,0,1,0,5,142125


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   LotArea       1460 non-null   int64
 1   BsmtFullBath  1460 non-null   int64
 2   BsmtHalfBath  1460 non-null   int64
 3   FullBath      1460 non-null   int64
 4   HalfBath      1460 non-null   int64
 5   TotRmsAbvGrd  1460 non-null   int64
 6   SalePrice     1460 non-null   int64
dtypes: int64(7)
memory usage: 80.0 KB


In [ ]:
dataset['BathRatio'] = (dataset['FullBath'] + 0.5 * dataset['HalfBath']) / dataset['TotRmsAbvGrd']
dataset['BedroomBathRatio'] = dataset['FullBath'] / (dataset['TotRmsAbvGrd'] - dataset['FullBath'])

x_train = dataset[['LotArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'BathRatio', 'BedroomBathRatio']]
y_train=dataset['SalePrice']

print(x_train.shape,y_train.shape)

(1460, 8) (1460,)


<ipython-input-146-0b6c7420367d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['BathRatio'] = (dataset['FullBath'] + 0.5 * dataset['HalfBath']) / dataset['TotRmsAbvGrd']
<ipython-input-146-0b6c7420367d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['BedroomBathRatio'] = dataset['FullBath'] / (dataset['TotRmsAbvGrd'] - dataset['FullBath'])


In [ ]:
scaler1 = preprocessing.StandardScaler()
scaler2 = preprocessing.MinMaxScaler()

x_train_scaled = scaler1.fit_transform(x_train)
y_train_scaled = scaler2.fit_transform(y_train.values.reshape(-1,1))

In [ ]:
l=linear_model.LinearRegression()
l.fit(x_train_scaled,y_train_scaled)

LinearRegression()

In [ ]:
param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1, 1, 2]
}
grid_search = GridSearchCV(l, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(x_train_scaled, y_train_scaled)

best_model = grid_search.best_estimator_

In [ ]:
data_test=pd.read_csv("/content/test.csv")
data_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [ ]:
data_test['BathRatio'] = (data_test['FullBath'] + 0.5 * data_test['HalfBath']) / data_test['TotRmsAbvGrd']
data_test['BedroomBathRatio'] = data_test['FullBath'] / (data_test['TotRmsAbvGrd'] - data_test['FullBath'])

x_test = data_test[['LotArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'BathRatio', 'BedroomBathRatio']]
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LotArea           1459 non-null   int64  
 1   BsmtFullBath      1457 non-null   float64
 2   BsmtHalfBath      1457 non-null   float64
 3   FullBath          1459 non-null   int64  
 4   HalfBath          1459 non-null   int64  
 5   TotRmsAbvGrd      1459 non-null   int64  
 6   BathRatio         1459 non-null   float64
 7   BedroomBathRatio  1459 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 91.3 KB


In [ ]:
for column in x_test:
  x_test[column].fillna(x_test[column].mean(), inplace=True)
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LotArea           1459 non-null   int64  
 1   BsmtFullBath      1459 non-null   float64
 2   BsmtHalfBath      1459 non-null   float64
 3   FullBath          1459 non-null   int64  
 4   HalfBath          1459 non-null   int64  
 5   TotRmsAbvGrd      1459 non-null   int64  
 6   BathRatio         1459 non-null   float64
 7   BedroomBathRatio  1459 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 91.3 KB


<ipython-input-172-2bd7093c7026>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column].fillna(x_test[column].mean(), inplace=True)


In [ ]:
x_test_scaled = scaler1.fit_transform(x_test)

In [ ]:
y_pred = best_model.predict(x_test_scaled)

In [ ]:
data_sub=pd.read_csv("/content/sample_submission.csv")
y_test=data_sub['SalePrice']
y_test_scaled = scaler2.fit_transform(y_test.values.reshape(-1,1))

In [ ]:
metrics.mean_squared_error(y_test_scaled, y_pred)

0.01839858602941437